# はじめに

データには多数の特徴やグループが存在するため、データがどのような構造になっているかを把握することは、モデリングを行う際に有利になります（ヒント：すべてのデータを同じように扱う大規模なモデルの構築を急がないでください）。

*バージョンの履歴とハイライト*

バージョン 01 : 最初のターゲット分析の追加とノートブックの公開

バージョン 02 : より多くの比較プロットを追加

バージョン 03 : 時間と資産の散布図と特徴に関する簡単な調査を追加

バージョン 04 : ターゲットと時間の関係についての詳細

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_feather('../input/training-data-to-feather-python-r-low-mem/train.feather')

# データの初見

観測数、アセット数、タイムステップ数の把握

In [ ]:
obs = train.shape[0]
print(f"number of observations: {obs}")

In [ ]:
time_steps, assets = train.time_id.nunique(), train.investment_id.nunique()
print(f"number of assets: {assets} \t time steps: {time_steps}")

In [ ]:
print(f"number of assets: {assets} (range from {train.investment_id.min()} to {train.investment_id.max()})")

アセット自体の数よりも、アセットの範囲の方が広がっています。実際、テストセットではアセットが一部変更されるため、トレーニングセットのものを一般化できるような戦略を考える必要がある。
チェック: https://www.kaggle.com/c/ubiquant-market-prediction/discussion/301693#1656092

# ターゲット分析

In [ ]:
obs_by_asset = train.groupby(['investment_id'])['target'].count()

fig, ax = plt.subplots(1, 1, figsize=(12, 6))
obs_by_asset.plot.hist(bins=60)
plt.title("target by asset distribution")
plt.show()

もっと注意深く、時間ごとの資産構成を見てみると、かなりの数の不連続性があり、その不連続性は時間の最初のセクションに多く存在することに気づくだろう。

In [ ]:
train[['investment_id', 'time_id']].plot.scatter('time_id', 'investment_id', figsize=(20, 30), s=0.5)
plt.show()

資産の分布の仕方は様々で、実際に観測される頻度が高い資産とそうでない資産がある。**良いCVとモデリング戦略はこれを考慮に入れるべきです** (サブサンプルを使っている場合は層別化する)。

In [ ]:
mean_target = train.groupby(['investment_id'])['target'].mean()
mean_mean_target = np.mean(mean_target)

fig, ax = plt.subplots(1, 1, figsize=(12, 6))
mean_target.plot.hist(bins=60)
plt.title("mean target distribution")
plt.show()

print(f"Mean of mean target: {mean_mean_target: 0.5f}")

資産別の平均目標値を見ると、ベル型の分布を示しているが、平均目標値がかなりマイナスの資産（-0.4エリア）とかなりプラスの資産（+0.8エリア）があるため、とにかく異常値があることに注意しよう。全体として、資産別の平均目標値はわずかにマイナス（-0.0231）である。

In [ ]:
sts_target = train.groupby(['investment_id'])['target'].std()
mean_std_target = np.mean(sts_target)

fig, ax = plt.subplots(1, 1, figsize=(12, 6))
sts_target.plot.hist(bins=60)
plt.title("standard deviation of target distribution")
plt.show()

print(f"Mean of std target: {mean_std_target: 0.5f}")

また、資産別の平均標準偏差（std）の平均は、興味深いパターンを示している。まず、右肩上がりになっており、一部の資産では標準偏差が大きくなっている（最大2.5）。一方、標準偏差がほぼゼロの資産もいくつか存在する。

In [ ]:
ax = sns.jointplot(x=obs_by_asset, y=mean_target, kind="reg", 
                   height=8, joint_kws={'line_kws':{'color':'red'}})
ax.ax_joint.set_xlabel('observations')
ax.ax_joint.set_ylabel('mean target')
plt.show()

資産別の観測量分布と資産別の目標値平均を併記すると、目標値が観測回数に比例してわずかに減少していることに気づくかもしれません。値の分散は観測回数が少ないほど大きくなる傾向があるので、今度は標準偏差を使って散布図を再プロットする必要があります。

In [ ]:
qx = sns.jointplot(x=obs_by_asset.values, y=sts_target, kind="reg", 
                   height=8, joint_kws={'line_kws':{'color':'red'}})
ax.ax_joint.set_xlabel('observations')
ax.ax_joint.set_ylabel('std target')
plt.show()

この散布図から、観測回数が少ないほど、平均的な目標の不確かさが大きいことがわかる。

**戦略**：トレーニングでは、オブザベーションの数を説明することによって、この効果をコントロールする必要があります。なぜなら、これは予測の不確実性を予測するからです。代わりにテストフェーズでは、あなたが知らない資産を扱っているとき、オブザベーションの平均数をインプットする必要があり、したがって、その資産の予測値の平均分散を期待します。

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
train.groupby('time_id')['investment_id'].nunique().plot()
plt.title("number of unique assets by time")
plt.show()

観測回数が少ない投資ほどリスクが高いことを説明したが、各時間ステップに存在する資産の数が全く異なり、しかも大きく振動していることに気がつく。利用可能時間が終了するまでに、資産の数は3分の1に増えている。

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(3, 1, 1,)
(train.groupby('time_id')['investment_id'].nunique()).plot()
plt.title("number of unique assets by time")

plt.subplot(3, 1, 2)
train.groupby('time_id')['target'].mean().plot()
plt.title("average target by time")
plt.axhline(y=mean_mean_target, color='r', linestyle='--', label="mean")
plt.legend(loc='lower left')

plt.subplot(3, 1, 3)
train.groupby('time_id')['target'].std().plot()
plt.title("std of target by time")
plt.axhline(y=mean_std_target, color='r', linestyle='--', label="mean")
plt.legend(loc='lower left')

plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=1.3, 
                    wspace=0.4, 
                    hspace=0.4)

plt.show()

In [ ]:
r = np.corrcoef(train.groupby('time_id')['investment_id'].nunique(), train.groupby('time_id')['target'].mean())[0][1]
print(f"Correlation of number of assets by target: {r:0.3f}")

時間ごとの資産数と時間ごとの平均目標値を並べてみると、資産数が少ないほど目標値の変動が大きく、目標値が高くなる傾向があることがわかる。実際、資産数と目標値の相関は負である。アセットアロケーション戦略をマーケットと一緒にモデル化しているのだろうか。

ここで、ターゲットと時間の関係をもう少し詳しく観察してみましょう。

In [ ]:
time2target_mean = train.groupby(['time_id'])['target'].mean()
time2target_std = train.groupby(['time_id'])['target'].std()

_, axes = plt.subplots(1, 1, figsize=(24, 12))
plt.fill_between(
        time2target_mean.index,
        time2target_mean - time2target_std,
        time2target_mean + time2target_std,
        alpha=0.1,
        color="b",
    )
plt.plot(
        time2target_mean.index, time2target_mean, "o-", color="b", label="Training score"
    )
plt.axhline(y=mean_mean_target, color='r', linestyle='--', label="mean")
axes.set_ylabel("target")
axes.set_xlabel("time")
plt.show()

明らかに対象はデトレンドされ、平均値ゼロ、単位標準偏差に強制されているが、資産数が減少すると平均値が変化し、その結果、信頼区間もシフトしてしまうことがある。このような平均値の変化のタイミングをテストセットで把握することができれば、非常に有利になる。

基本的にはこのチャートがポイントになります。競技の課題は、ある資産の1日のポジションを知ることです。その資産は平均に近いのか、それともどれくらい離れているのか（基本的にはボラティリティを予測していることになる）。実際には、評価は各時間IDのピアソン相関係数の平均値に基づいています。

次のチャートでは、資産70のターゲットを市場平均と単位標準偏差帯でオーバーインポーズしています。

明らかに資産70のポジションはそのパフォーマンスに依存しますが、その期間IDの平均と標準偏差の計算方法にも依存します（投資のバスケット内部のボラティリティを分析しているのでしょうか、多分？）

In [ ]:
time2target_mean = train.groupby(['time_id'])['target'].mean()
time2target_std = train.groupby(['time_id'])['target'].std()

_, axes = plt.subplots(1, 1, figsize=(24, 12))
plt.fill_between(
        time2target_mean.index,
        time2target_mean - time2target_std,
        time2target_mean + time2target_std,
        alpha=0.1,
        color="b",
    )
plt.plot(
        time2target_mean.index, time2target_mean, "o-", color="b", label="Training score"
    )
plt.axhline(y=mean_mean_target, color='r', linestyle='--', label="mean")

asset = 70
plt.plot(train[train.investment_id==asset].time_id,
               train[train.investment_id==asset].target, '.')

axes.set_ylabel("target")
axes.set_xlabel("time")
plt.show()

**戦略**：CVの戦略は明確で、time_idでgroupkfoldを行い、すべてのアセットをtime_idに実在させるか、trainまたはvalidationで維持する必要があります。

# 仮説：特徴量から資産別のオブザーバ数の代理を得ることができる？

In [ ]:
obs_by_asset = train.groupby(['investment_id'])['target'].count().to_dict()
target = train.investment_id.copy().replace(obs_by_asset).astype(np.int16)
features = train.columns[4:]

del(obs_by_asset)

In [ ]:
corrs = list()
for col in features:
    corr = np.corrcoef(target, train[col])[0][1]
    corrs.append(corr)
    
del(target)

In [ ]:
feat_importances = pd.Series(corrs, index=features)
feat_importances.nlargest(20).plot(kind='barh', figsize=(12, 6)).invert_yaxis()
plt.show()

実際にいくつかの特徴量に基づいて、ある資産の観測値が少ないか多いかという事実は、高い相関を持つ特徴量に基づいてかなり予測できるはずです。

## もしノートブックが好きで、それが有用だと分かったら、upvote :-) を検討してください。